In [1]:
!pip install transformers
!pip install --upgrade tensorflow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from transformers import InputExample, InputFeatures
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

2022-04-15 06:27:54.508555: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-15 06:27:54.508594: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [26]:
import tensorflow as tf
import pandas as pd
df = pd.read_json('review_10000.json', lines=True)

#get only the text
#sentences = df.text.values

#assigns positive of negative (words) to sentiment
df["Sentiment"] = df.stars.apply(lambda score: "positive" if score >= 3 else "negative")
# change positive / negative to 1/ 0
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})
df['ratings'] = df.stars
#just show the review and the new 1/0 
df = df[["text", "Sentiment", "ratings"]]

print(df)

                                                   text  Sentiment  ratings
0     If you decide to eat here, just be aware it is...          1        3
1     I've taken a lot of spin classes over the year...          1        5
2     Family diner. Had the buffet. Eclectic assortm...          1        3
3     Wow!  Yummy, different,  delicious.   Our favo...          1        5
4     Cute interior and owner (?) gave us tour of up...          1        4
...                                                 ...        ...      ...
9995  Excellent food and service. The place is funct...          1        5
9996  Just about to get tucked into a meatloaf that ...          1        5
9997  Outstanding customer service! And my car is dr...          1        5
9998  I and my husband went here for Dinner one day ...          1        5
9999  I saw Big Data when they came to town at the K...          0        2

[10000 rows x 3 columns]


In [27]:
reviews = df.text.values.tolist()
labels = df.Sentiment.tolist()
stars = df.ratings.tolist()



In [28]:
print(reviews[:2])
print(labels[:2])

["If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", "I've taken a lot of spin classes over the years, and nothing compares to the classes at Body Cycle. From the nice, clean space and amazing bikes, to the welcoming and motivating instructors, every class is a top notch work out.\n\nFor anyone who struggles to fit workouts in, the online scheduling system makes it easy to plan ahead (and there's no need to line up way in advanced like many gyms make you do).\n\nThere is no way I can write this review without giving Russell, th

In [29]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [30]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


In [31]:
tokenizer([training_sentences[0]], truncation=True,
                            padding=True, max_length=128)

{'input_ids': [[101, 1045, 3641, 7975, 14841, 15714, 16137, 7911, 1998, 3869, 15478, 2005, 4596, 1012, 2027, 2134, 1005, 1056, 5510, 2004, 4840, 1998, 2204, 2004, 1045, 2031, 3517, 1012, 2174, 1010, 1996, 3954, 1997, 4825, 2170, 2067, 2157, 2185, 1998, 17806, 1012, 1045, 2001, 7622, 2011, 1996, 4825, 1005, 1055, 2064, 7983, 1998, 29454, 29206, 3401, 1999, 8013, 9967, 1012, 2947, 1010, 1045, 2228, 1045, 2572, 2183, 2000, 2175, 2067, 1998, 3046, 2009, 2153, 1999, 1037, 2261, 3134, 2000, 2156, 3251, 2045, 2003, 2151, 7620, 1999, 1996, 3737, 1997, 2833, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [32]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [33]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))


In [34]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use i

In [35]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [36]:
model.save_pretrained("./sentiment")
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("./sentiment")


Some layers from the model checkpoint at ./sentiment were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./sentiment and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
totalPos = 0
totalNeg = 0

for i in stars:
    if (i >= 3):
        totalPos = totalPos + 1
    else: 
        totalNeg = totalNeg + 1
# count the number of positives and negatives based on numbers
print(len(reviews))
print(totalPos)
print(totalNeg)

10000
8158
1842


In [ ]:
labelString = ['Negative','Positive']

# test each review
# middle_index=len(reviews)//2

# first_half=reviews[:middle_index]
# sec_half=reviews[middle_index:]
# label_first_half = labels[:middle_index]
# label_sec_half = labels[middle_index:]
index = 0
currPos = 0
currNeg = 0
for item1 in reviews:
    # compare to item1 in second half
    predict_input = tokenizer.encode(item1,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
   
    tf_output = loaded_model.predict(predict_input)[0]
    tf_prediction = tf.nn.softmax(tf_output, axis=1)
    label = tf.argmax(tf_prediction, axis=1)
    label = label.numpy()
    
    if (label[0] == 0):
        currNeg = currNeg + 1
    else:
        currPos = currPos + 1
    index = index + 1
    

print(index)
print(currPos)
print(currNeg)
 